In [1]:
import os
import openai

In [2]:
# Read API Key
with open('API_KEY.txt') as f:
    api_key = f.readlines()[0]

# Set API Key
openai.api_key = api_key

## Automatic Exam Creator

In [3]:
def create_test_prompt(topic, n_questions, n_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {n_questions} questions. "\
           + f"Each question should have {n_possible_answers} options. "\
           + f"Also include the correct answer for each question using the starting string 'Correct Answer: ' "\
           + "If you are unsure about the correct answer to a question, generate a new pair of question and options. " # Prompt Engineering
    return prompt

In [4]:
# Parameters
TOPIC = "Jabra Panacast50"  # "US History", "Mathematics", "Deep Learning", "Python Programming"
# TOPIC = "Computer Vision"
NUM_QUESTIONS = 4
NUM_POSSIBLE_ANSWERS = 4

In [5]:
# TEST
create_test_prompt(TOPIC, NUM_QUESTIONS, NUM_POSSIBLE_ANSWERS)

"Create a multiple choice quiz on the topic of Jabra Panacast50 consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer: ' If you are unsure about the correct answer to a question, generate a new pair of question and options. "

In [6]:
# API Request
response = openai.Completion.create(engine='text-davinci-003',
                                    prompt=create_test_prompt(TOPIC, NUM_QUESTIONS, NUM_POSSIBLE_ANSWERS),
                                    max_tokens=256,
                                    temperature=0.7)

In [7]:
response

<OpenAIObject text_completion id=cmpl-6wxthnTdyPQGqComOTEfJV26qkVDe at 0x1cfeacd5cb0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nQ1. What type of camera does the Jabra Panacast50 use?\nA. HD\nB. IP\nC. 4K\nD. USB\nCorrect Answer: C. 4K\n\nQ2. What type of wireless technology does the Jabra Panacast50 use?\nA. Bluetooth\nB. Wi-Fi\nC. RF\nD. NFC\nCorrect Answer: B. Wi-Fi\n\nQ3. How many microphones are included with the Jabra Panacast50?\nA. 2\nB. 4\nC. 6\nD. 8\nCorrect Answer: A. 2\n\nQ4. What is the microphone sensitivity of the Jabra Panacast50?\nA. -90dB\nB. -80dB\nC. -70dB\nD. -60dB\nCorrect Answer: D. -60dB"
    }
  ],
  "created": 1679511289,
  "id": "cmpl-6wxthnTdyPQGqComOTEfJV26qkVDe",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 185,
    "prompt_tokens": 66,
    "total_tokens": 251
  }
}

In [8]:
print(response['choices'][0]['text'])



Q1. What type of camera does the Jabra Panacast50 use?
A. HD
B. IP
C. 4K
D. USB
Correct Answer: C. 4K

Q2. What type of wireless technology does the Jabra Panacast50 use?
A. Bluetooth
B. Wi-Fi
C. RF
D. NFC
Correct Answer: B. Wi-Fi

Q3. How many microphones are included with the Jabra Panacast50?
A. 2
B. 4
C. 6
D. 8
Correct Answer: A. 2

Q4. What is the microphone sensitivity of the Jabra Panacast50?
A. -90dB
B. -80dB
C. -70dB
D. -60dB
Correct Answer: D. -60dB


## Exam Simulator

In [9]:
# Extract Question & Options
def create_student_view(response, n_questions):
    student_view = {1: ''}
    question_number = 1
    # Split response by Questions and Options
    for line in response.split("\n"):
        if not line.startswith("Correct Answer:"):
            # If not correct answer, it's still a question and option line
            student_view[question_number] += line + "\n"
        else:
            if question_number < n_questions:
                question_number += 1
                student_view[question_number] = ''
    return student_view

In [10]:
result = create_student_view(response['choices'][0]['text'], NUM_QUESTIONS)

In [11]:
# Extract just the Question & Options
for key in result:
    print(result[key])



Q1. What type of camera does the Jabra Panacast50 use?
A. HD
B. IP
C. 4K
D. USB


Q2. What type of wireless technology does the Jabra Panacast50 use?
A. Bluetooth
B. Wi-Fi
C. RF
D. NFC


Q3. How many microphones are included with the Jabra Panacast50?
A. 2
B. 4
C. 6
D. 8


Q4. What is the microphone sensitivity of the Jabra Panacast50?
A. -90dB
B. -80dB
C. -70dB
D. -60dB



In [12]:
# Extract Correct Answers
def extract_answers(response, n_questions):
    answers = {1: ''}
    question_number = 1
    # Split response
    for line in response.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line + "\n"
            if question_number < n_questions:
                question_number += 1
                answers[question_number] = ''
    return answers

In [13]:
answers = extract_answers(response['choices'][0]['text'], NUM_QUESTIONS)
answers

{1: 'Correct Answer: C. 4K\n',
 2: 'Correct Answer: B. Wi-Fi\n',
 3: 'Correct Answer: A. 2\n',
 4: 'Correct Answer: D. -60dB\n'}

## Take & Grade Exam

In [14]:
student_view = create_student_view(response['choices'][0]['text'], NUM_QUESTIONS)

In [15]:
answers = extract_answers(response['choices'][0]['text'], NUM_QUESTIONS)

In [16]:
# Take Exam
def take_exam(student_view):
    student_answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        student_answers[question] = answer
    return student_answers

In [18]:
student_answers = take_exam(student_view)



Q1. What type of camera does the Jabra Panacast50 use?
A. HD
B. IP
C. 4K
D. USB



Enter your answer:  d



Q2. What type of wireless technology does the Jabra Panacast50 use?
A. Bluetooth
B. Wi-Fi
C. RF
D. NFC



Enter your answer:  a



Q3. How many microphones are included with the Jabra Panacast50?
A. 2
B. 4
C. 6
D. 8



Enter your answer:  d



Q4. What is the microphone sensitivity of the Jabra Panacast50?
A. -90dB
B. -80dB
C. -70dB
D. -60dB



Enter your answer:  d


In [19]:
student_answers

{1: 'd', 2: 'a', 3: 'd', 4: 'd'}

In [20]:
answers

{1: 'Correct Answer: C. 4K\n',
 2: 'Correct Answer: B. Wi-Fi\n',
 3: 'Correct Answer: A. 2\n',
 4: 'Correct Answer: D. -60dB\n'}

In [21]:
answers[1][16].lower()

'c'

In [22]:
def grade_exam(correct_answers, student_answers):
    correct = 0
    for question, answer in student_answers.items():
        # 
        if answer == correct_answers[question][16].lower():
            correct += 1
    
    grade = 100 * correct / len(answers)
    
    if grade < 60:
        passed = "NO PASS"
    else:
        passed = "PASS"
    
    return f"{correct} / {len(answers)} are correct! You got {grade} % grade, {passed}"

In [23]:
print(grade_exam(answers, student_answers))

1 / 4 are correct! You got 25.0 % grade, NO PASS
